In [1]:
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils


import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 


import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import copy
import numpy as np

In [3]:
df = MyPyTable()
df.load_from_file("high_diamond_ranked_10min.csv")
y = df.get_column("blueWins")

X = copy.deepcopy(df.data)
# randomizing indexes to sort out a stratified sample
X_indexes = [ i for i in range(len(X))]
myutils.randomize_in_place(X_indexes,y,0)
grouped_X =[]
grouped_y = []
for i in range(len(X)):
    if grouped_y.count(y[i]) >0:
        grouped_X[grouped_y.index(y[i])].append(X_indexes[i])
    else:
        grouped_y.append(y[i])
        grouped_X.append([X_indexes[i]])
X_data = []
y_data = []
for i in range(1500):
    for j in range(len(grouped_X)):
        X_data.append(X[grouped_X[j][i]])
        y_data.append(grouped_y[j])

X = copy.deepcopy(X_data)

y = copy.deepcopy(y_data)
for entry in X:
    del entry[0]
    del entry[0]
knn_clf = MyKNeighborsClassifier()
nb_clf = MyNaiveBayesClassifier()
dummy_clf = MyDummyClassifier()
tree_clf = MyDecisionTreeClassifier()

In [3]:
myutils.compute_clf_stats(knn_clf,X,y,"kNN Classifier")
myutils.compute_clf_stats(dummy_clf,X,y,"Dummy Classifier")


Hold-Out Method
kNN Classifier : accuracy =  0.6909090909090909  error =  0.3090909090909091
binary precision:  0.6915322580645161
binary recall:  0.6915322580645161
f1 score:  0.6915322580645161
Confusion Matrix
[[341, 153], [153, 343]]

Hold-Out Method
Dummy Classifier : accuracy =  0.5  error =  0.5
binary precision:  0.0
binary recall:  0.0
f1 score:  0
Confusion Matrix
[[495, 0], [495, 0]]


In [4]:
header = df.column_names.copy()
del header[0:2]
del header[-9:-1]
del header[-1]
del header[10:15]
del header[12:14]
print(header)

['blueWardsPlaced', 'blueWardsDestroyed', 'blueFirstBlood', 'blueKills', 'blueDeaths', 'blueAssists', 'blueEliteMonsters', 'blueDragons', 'blueHeralds', 'blueTowersDestroyed', 'blueGoldDiff', 'blueExperienceDiff', 'redWardsPlaced', 'redWardsDestroyed', 'redFirstBlood', 'redKills', 'redDeaths', 'redAssists', 'redEliteMonsters', 'redDragons', 'redHeralds', 'redTowersDestroyed']


In [5]:
X = copy.deepcopy(X_data)

for entry in X:
    del entry[0:2]
    del entry[-9:-1]
    del entry[-1]
    del entry[10:15]
    del entry[12:14]

myutils.compute_clf_stats(knn_clf,X,y,"kNN Classifier")
myutils.compute_clf_stats(dummy_clf,X,y,"Dummy Classifier")


Hold-Out Method
kNN Classifier : accuracy =  0.6525252525252525  error =  0.3474747474747475
binary precision:  0.6583333333333333
binary recall:  0.6370967741935484
f1 score:  0.6475409836065574
Confusion Matrix
[[316, 180], [164, 330]]

Hold-Out Method
Dummy Classifier : accuracy =  0.5  error =  0.5
binary precision:  0.0
binary recall:  0.0
f1 score:  0
Confusion Matrix
[[495, 0], [495, 0]]


In [4]:
# equal width binning
X = copy.deepcopy(X_data)
y = [str(value) for value in y]
for entry in X:
    del entry[0:2]
    del entry[-9:-1]
    del entry[-1]
    del entry[10:15]
    del entry[12:14]
for j in range(len(X[0])):
    binned_col = myutils.binning([row[j] for row in X])
    for i in range(len(X)):
        X[i][j] = binned_col[i]

In [5]:
X_train, X_test, y_train,y_test = myevaluation.train_test_split(X,y,random_state=0)

forest_clf = MyRandomForestClassifier()
forest_clf.fit(X_train,y_train,5,3,3)
y_pred_clf = forest_clf.predict(X_test)

accuracy = myevaluation.accuracy_score(y_test,y_pred_clf)
print("Hold-Out Method")
print("Random Forest : accuracy = ", accuracy, " error = ", 1 - accuracy)

Hold-Out Method
Random Forest : accuracy =  0.6575757575757576  error =  0.3424242424242424


In [6]:
X_train, X_test, y_train,y_test = myevaluation.train_test_split(X,y,random_state=0)

forest_clf = MyRandomForestClassifier()
forest_clf.fit(X_train,y_train,20,7,3)
y_pred_clf = forest_clf.predict(X_test)

accuracy = myevaluation.accuracy_score(y_test,y_pred_clf)
print("Hold-Out Method")
print("Random Forest : accuracy = ", accuracy, " error = ", 1 - accuracy)

KeyboardInterrupt: 

In [5]:
X_train, X_test, y_train,y_test = myevaluation.train_test_split(X,y,random_state=0)

forest_clf = MyRandomForestClassifier()
forest_clf.fit(X_train,y_train,5,3,4)
y_pred_clf = forest_clf.predict(X_test)

accuracy = myevaluation.accuracy_score(y_test,y_pred_clf)
print("Hold-Out Method")
print("Random Forest : accuracy = ", accuracy, " error = ", 1 - accuracy)

predicting
predicting
predicting
predicting
predicting


KeyboardInterrupt: 

## EDA

The following are some graphs that help us identify some interesting relationships. Starting with sorting the table into TWO tables with all the data of blue wins and all the data with red wins.

In [ ]:
data = MyPyTable()
data.load_from_file('high_diamond_ranked_10min.csv')
blue_wins_table = []
blue_loses_table = []

for row in data.data:
    if row[1] == 1: # blue wins
        blue_wins_table.append(row)
    elif row[1] == 0: # red wins
        blue_loses_table.append(row)

print(data.column_names)
blue_wins_df = MyPyTable(column_names=data.column_names, data=blue_wins_table)
blue_loses_df = MyPyTable(column_names=data.column_names, data=blue_loses_table)
print(blue_wins_df.column_names)

## Graphing Blue Stats VS Red Stats when Blue WINS

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
blue_avg_level = blue_wins_df.get_column('blueAvgLevel')
red_avg_level = blue_wins_df.get_column('redAvgLevel')

plt.plot(blue_avg_level, red_avg_level, "b.")
plt.title('Avg Blue Lv. VS Avg. Red Level (When Blue WINS)')
plt.xlabel('Avg. Blue Lvl')
plt.ylabel('Av.g Red Lvl')
plt.show()

In [ ]:
blue_exp = blue_wins_df.get_column('blueTotalExperience')
red_exp = blue_wins_df.get_column('redTotalExperience')

plt.plot(blue_exp, red_exp, "b.")
plt.title('Blue Experience VS Red Experience (When Blue WINS)')
plt.xlabel('Blue Experience')
plt.ylabel('Red Experience')
plt.show()

In [ ]:
blue_gold = blue_wins_df.get_column('blueTotalGold')
red_gold = blue_wins_df.get_column('redTotalGold')

plt.plot(blue_gold, red_gold, "b.")
plt.title('Blue Gold VS Avg. Red Gold (When Blue WINS)')
plt.xlabel('Blue Gold')
plt.ylabel('Red Gold')
plt.show()

## Does Getting the First Kill Matter?

In [ ]:
x = [1,2]
counts = [0,0]
for row in blue_wins_df.data:
    if row[4] == 1:
        counts[0] += 1
    elif row[4] == 0:
        counts[1] += 1

plt.bar(np.arange(len(x)), counts)
plt.xticks(np.arange(len(x)), ['Got First Kill', 'Did NOT Get First Kill'])
plt.ylabel('Counts')
plt.title('Whether Blue Got the First Kill When Blue WINS')
plt.show()

In [ ]:
x = [1,2]
counts = [0,0]
for row in blue_loses_df.data:
    if row[23] == 1:
        counts[0] += 1
    elif row[23] == 0:
        counts[1] += 1

plt.bar(np.arange(len(x)), counts)
plt.xticks(np.arange(len(x)), ['Got First Kill', 'Did NOT Get First Kill'])
plt.ylabel('Counts')
plt.title('Whether Red Got the First Kill When RED WINS')
plt.show()

## Calculating and Plotting Kill/Death Ratio

In [ ]:
print(blue_wins_df.column_names)

In [ ]:
header = blue_wins_df.column_names
header.append(['BlueKillDeathRatio', 'RedKillDeathRatio'])

blue_wins_df.column_names = header
blue_loses_df.column_names = header

for row in blue_wins_df.data:
    if row[6] == 0:
        if row[25] == 0:
            row.append([0,0])
        else:
            row.append([0, row[24]/row[25]])

    if row[25] == 0:
        row.append([row[5]/row[6],0])

for row in blue_loses_df.data:
    if row[6] == 0:
        if row[25] == 0:
            row.append([0,0])
        else:
            row.append([0, row[24]/row[25]])

    if row[25] == 0:
        row.append([row[5]/row[6],0])